In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
# Matching dictionary for team names between events and tracking data
TEAM_NAMES = {'Canada': 'Olympic (Women) - Canada',
              'USA': 'Olympic (Women) - United States',
              'Finland': 'Olympic (Women) - Finland',
              'ROC': 'Olympic (Women) - Olympic Athletes from Russia',
              'Switzerland': 'Olympic (Women) - Switzerland'}

In [3]:
tracking_dir = "external/Big-Data-Cup-2022/data"
pbp_dir = "external/Big-Data-Cup-2022/data"
play_by_play_data_file = 'pxp_womens_oly_2022_v2.csv'
power_play_info_file = 'pp_info.csv'

In [4]:
def sort_row_by_col(row, col="player_name"):
    if row[col + "_2"] < row[col]:
        for col in ["player_name", "jn", "position"]:
        # for col in ["player_name"]:
            tmp = row[col + "_2"]
            row[col + "_2"] = row[col]
            row[col] = tmp
        
    return row

In [5]:
play_by_play_data = pd.read_csv(os.path.join(tracking_dir, play_by_play_data_file))

In [6]:
def run(game, game_date, team_name):
    this_game = play_by_play_data.copy()
    # power_play_info = pd.read_csv(os.path.join(pbp_dir, power_play_info_file))
    roster_info = pd.read_csv(os.path.join(tracking_dir, game, game + " roster.csv"), index_col=0)
    
    this_game = this_game[this_game["game_date"] == game_date]
    this_game = this_game[this_game["team_name"] == team_name]
    this_game = this_game[(this_game["event"] == "Play") & this_game["event_successful"]]
    this_game = this_game.join(roster_info, on = "player_name").join(roster_info, on = "player_name_2", rsuffix="_2")
    # Exclude goalie
    # this_game = this_game[(this_game["position"] != "Goalie") & (this_game["position_2"] != "Goalie")]

    # By player
    # collected_passes = this_game.apply(lambda row: sort_row_by_col(row, col="player_name"), axis=1)
    # collected_passes = collected_passes.loc[:, ["player_name", "player_name_2"]].value_counts()
    # for period in [1, 2, 3]:
        # collected_passes = this_game[this_game["period"] == period]
    
    # By position
    collected_passes = this_game.apply(lambda row: sort_row_by_col(row, col="position"), axis=1)
    # Remove loops
    # collected_passes = collected_passes[collected_passes["position"] != collected_passes["position_2"]]
    collected_passes = collected_passes.loc[:, ["position", "position_2"]].value_counts()
    print(collected_passes)
    
    # Write to CSV
    collected_passes.to_csv(f"{game} passes.csv")

In [7]:
game = "2022-02-16 Switzerland at Finland"
game_date = "16/2/2022"
team_name = TEAM_NAMES["Finland"]
run(game, game_date, team_name)

position       position_2   
Left Defense   Right Defense    59
Left Wing      Right Defense    57
Left Defense   Left Wing        45
Left Wing      Right Wing       41
Left Defense   Right Wing       39
Center         Left Defense     28
               Left Wing        24
Right Defense  Right Wing       24
Center         Right Defense    23
               Right Wing       21
Left Wing      Left Wing        21
Center         Center            6
Right Wing     Right Wing        6
Goalie         Left Defense      4
Left Defense   Left Defense      3
Right Defense  Right Defense     3
Goalie         Right Defense     2
               Left Wing         2
Name: count, dtype: int64
